## Extração dos dados

In [1]:
# importando o pacotes

import pandas as pd
import numpy as np
import sqlite3

In [2]:
# conectando ao banco de dados - produção

connect_db_imoveis = sqlite3.connect('../../data/imoveis_prod.db')

In [3]:
# definição da query final

query = r'''
    SELECT cidade.nome as "cidade",
    estado.nome as "estado",
    imoveis.area,
    imoveis.num_quartos,
    imoveis.num_banheiros,
    imoveis.num_andares,
    imoveis.garagem,
    imoveis.aceita_animais,
    imoveis.mobilia,
    imoveis.valor_aluguel,
    imoveis.valor_condominio,
    imoveis.valor_iptu,
    imoveis.valor_seguro_incendio
    FROM cidade INNER JOIN estado
    ON cidade.codigo_estado = estado.codigo
    INNER JOIN imoveis
    ON cidade.codigo = imoveis.codigo_cidade;
    ''' 

In [4]:
# criando um dataframe com esta query

df = pd.read_sql_query(query, connect_db_imoveis)
df.head(6)

,cidade,estado,area,num_quartos,num_banheiros,num_andares,garagem,aceita_animais,mobilia,valor_aluguel,valor_condominio,valor_iptu,valor_seguro_incendio
0,São Paulo,SP,70,2,1,7,1,acept,furnished,3300,2065,211,42
1,São Paulo,SP,320,4,4,20,0,acept,not furnished,4960,1200,1750,63
2,Porto Alegre,RS,80,1,1,6,1,acept,not furnished,2800,1000,0,41
3,Porto Alegre,RS,51,2,1,2,0,acept,not furnished,1112,270,22,17
4,São Paulo,SP,25,1,1,1,0,not acept,not furnished,800,0,25,11
5,São Paulo,SP,376,3,3,-,7,acept,not furnished,8000,0,834,121


In [5]:
df.shape

(10692, 13)

In [6]:
df.dtypes

cidade                   object
estado                   object
area                     object
num_quartos              object
num_banheiros            object
num_andares              object
garagem                  object
aceita_animais           object
mobilia                  object
valor_aluguel            object
valor_condominio         object
valor_iptu               object
valor_seguro_incendio    object
dtype: object

In [7]:
df.isnull().sum()

cidade                   0
estado                   0
area                     0
num_quartos              0
num_banheiros            0
num_andares              0
garagem                  0
aceita_animais           0
mobilia                  0
valor_aluguel            0
valor_condominio         0
valor_iptu               0
valor_seguro_incendio    0
dtype: int64

## Data transformation/cleaning

In [8]:
# convrertendo os valores para numérico

from numpy import int64, float64

df['area'] = df['area'].astype(int64)
df['num_quartos'] = df['num_quartos'].astype(int64)
df['num_banheiros'] = df['num_banheiros'].astype(int64)
df['garagem'] = df['garagem'].astype(int64)
df['valor_aluguel'] = df['valor_aluguel'].astype(float64)
df['valor_condominio'] = df['valor_condominio'].astype(float64)
df['valor_iptu'] = df['valor_iptu'].astype(float64)
df['valor_seguro_incendio'] = df['valor_seguro_incendio'].astype(float64)

### Importante
**Apesar de não reconhecer missing values,o num_andares não pôde ser convertido pois encontrou caracteres.**

In [9]:
# encontrando e substituindo caracteres

df['num_andares'].unique()

array(['7', '20', '6', '2', '1', '-', '4', '3', '10', '11', '24', '9',
       '8', '17', '18', '5', '13', '15', '16', '14', '26', '12', '21',
       '19', '22', '27', '23', '35', '25', '46', '28', '29', '301', '51',
       '32'], dtype=object)

In [10]:
# substituindo - pelo menor valor

df['num_andares'].replace({'-': 1}, inplace=True)

In [11]:
# convertendo

df['num_andares'] = df['num_andares'].astype(int64)

In [12]:
# verificando os valores

df.dtypes

cidade                    object
estado                    object
area                       int64
num_quartos                int64
num_banheiros              int64
num_andares                int64
garagem                    int64
aceita_animais            object
mobilia                   object
valor_aluguel            float64
valor_condominio         float64
valor_iptu               float64
valor_seguro_incendio    float64
dtype: object

In [13]:
# encoder de animais e mobilia

print(df['aceita_animais'].unique())
print(df['mobilia'].unique())

['acept' 'not acept']
['furnished' 'not furnished']


In [14]:
# substituindo
animais_map = {'not acept' : 0, 'acept': 1}
mobilia_map = {'not furnished' : 0, 'furnished': 1}

df['aceita_animais'] = df['aceita_animais'].astype(str).map(animais_map)
df['mobilia'] = df['mobilia'].astype(str).map(mobilia_map)

In [15]:
df.dtypes

cidade                    object
estado                    object
area                       int64
num_quartos                int64
num_banheiros              int64
num_andares                int64
garagem                    int64
aceita_animais             int64
mobilia                    int64
valor_aluguel            float64
valor_condominio         float64
valor_iptu               float64
valor_seguro_incendio    float64
dtype: object

In [16]:
# Limpando cidade e estado

print(df['cidade'].unique())
print(df['estado'].unique())

['São Paulo' 'Porto Alegre' 'Rio de Janeiro' 'Campinas']
['SP' 'RS' 'RJ']


In [17]:
# colocando cada cidade em maiúsculo

df['cidade'] = df['cidade'].str.title()

In [18]:
df['cidade'].unique()

array(['São Paulo', 'Porto Alegre', 'Rio De Janeiro', 'Campinas'],
      dtype=object)

## Salvando o dataset final

In [20]:
df.to_csv('final_df.csv', index=False)